# State of North Dakota: Approachable AI Hands On

In this demonstration, we will leverage several open-source python libraries to train, text and validate machine learning models

We will introduce several commonly used, and well documented, data science packages:
- Pandas - a popular data manipulation package
- numpy - a package for accelerated mathematical operations on arrays
- scikit-learn - allows for a range of machine learning models to be used with a common interface
- matplotlib - pythons standard graph plotting package
- seaborn - more advanced plotting, when installed alongside matplotlib improves the graphs


In [ ]:
# Put several standard import statements here - we will import specific scikit-learn packages later on as they are needed

# Packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# Preprocessing components
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import StandardScaler

# Models
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor

warnings.filterwarnings("ignore")
%matplotlib inline

In [ ]:
# Let's put in some congiguration variables here
data_path = "../data/forestfires.csv"

Column definitions are as follows:

1. X - x-axis spatial coordinate within the Montesinho park map: 1 to 9
2. Y - y-axis spatial coordinate within the Montesinho park map: 2 to 9
3. month - month of the year: 'jan' to 'dec'
4. day - day of the week: 'mon' to 'sun'
5. FFMC - FFMC index from the FWI system: 18.7 to 96.20
6. DMC - DMC index from the FWI system: 1.1 to 291.3
7. DC - DC index from the FWI system: 7.9 to 860.6
8. ISI - ISI index from the FWI system: 0.0 to 56.10
9. temp - temperature in Celsius degrees: 2.2 to 33.30
10. RH - relative humidity in %: 15.0 to 100
11. wind - wind speed in km/h: 0.40 to 9.40
12. rain - outside rain in mm/m2 : 0.0 to 6.4
13. area - the burned area of the forest (in ha): 0.00 to 1090.84
(this output variable is very skewed towards 0.0, thus it may make
sense to model with the logarithm transform).

### Load the dataset and do a few simple explorations

In [ ]:
df = pd.read_csv(data_path)

In [ ]:
# The head method shows the first few lines of the data, 5 by default
df.head()

In [ ]:
# Let's also see how many null values we have in our dataset
df.info()

In [ ]:
# The describe method gives us summary statistics of all of the numeric columns in the dataframe
df.describe()

In [ ]:
# We can see how correlated different columns might be with each other 
# note: a variable is always perfectly correlated with itself!
df.corr()

In [ ]:
# And we can actually plot these relationships with seaborn
sns.pairplot(df[["FFMC", "DMC", "DC", "ISI", "temp", "RH", "wind", "rain", "area"]])
plt.show()

### Now we will perform some dataset preparation steps on the dataframe

In [ ]:
# Remove any duplicate rows - the inplace = True will prevent python making a copy of the dataframe
df.drop_duplicates(inplace=True)

In [ ]:
# We will remove the day column from the dataset
df.drop(labels="day", axis=1, inplace=True)

In [ ]:
# Perform indicator encoding on the month columns
df = pd.get_dummies(df, columns=["month"])

# note - this operation automatically removes the original column to avoid downstream issues

In [ ]:
# Split the data into a training set and a testing set
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

### Now we are going to perform dataset preparation steps that must be performed only on the training set

In [ ]:
# Deal with missing data in the DC column of the  training set by imputing with the mean

# Instantiate an imputer
imp = Imputer(missing_values=np.NaN, strategy="mean")

# Fit the imputer - this will ensure that both train and test data are fitted with the precise same mean
imp.fit(df_train[["DC"]])

# Perform the imputation on the column - note you only need ravel if there is one column
df_train["DC"] = imp.transform(df_train[["DC"]]).ravel()

In [ ]:
# Deal with any other missing data by deleting the entire row 
df_train.dropna(inplace=True)

In [ ]:
# Remove outliers
# There are many ways of performing automatic outlier detection, here we will just do it with a threshold on FFMC

def outlier_removal(dframe, threshold):
    """
    Function to perform outlier removal on a pandas dataframe with the FFMC column
    """
    # Create a boolean mask of all values that are  outside the expected range
    rows_to_remove = dframe["FFMC"] > threshold

    # Remove the rows that the Boolean mask picked out
    return dframe.drop(dframe[rows_to_remove].index)

df_train = outlier_removal(df_train, 120)

In [ ]:
# Normalize the DC column using the z-transformation - called StandardScaler in SK-learn

# Instantiate a standard scaler
normalizer = StandardScaler()

# Fit the standard scaler on the training data
normalizer.fit(df_train[["DC"]])

# Perform the normalization - note you only need ravel if there is only one column
df_train["DC"] = normalizer.transform(df_train[["DC"]]).ravel()

In [ ]:
# Engineer an extra column - this extra column is an indicator whether it rained or not

def add_did_it_rain(dframe):
    """
    Add an extra Boolean indicator column indicating whetherit rained or not that day.
    
    Note that the condition returns the negative condition, so we set the "other" to the positive condition
    """
    dframe["did_it_rain"] = dframe["rain"].where(dframe["rain"] == 0.0, other = 1)
    return dframe

df_train = add_did_it_rain(df_train)

In [ ]:
df_train.shape

In [ ]:
df_train.head()

### Let's build a linear regession model on the training set, and then run it on the test set

This method will serve well to illustrate how to build a single machine learning model

In [ ]:
# Define training feature matrix and target vector
X_train = df_train.drop("area", axis=1)
y_train = df_train["area"]

In [ ]:
# Instantiate a linear regression model - use n_jobs to parallelize the calculations
model = LinearRegression(n_jobs=-1, fit_intercept=True)

# Fit the model
model.fit(X_train, y_train)

# Print the beta-parameters and the y-intercept
print("Model parameters: {}".format(model.coef_))
print("Model y-intercept: {:.2f}".format(model.intercept_))
print("Training R^2 parameter: {:.4f}".format(model.score(X_train, y_train)))

In [ ]:
# We now need to prepare the test set using the exact same method as we used to prepare the training set

# Use our previouly created imputer to fill in the NaNs with the average from the training set

# Perform the imputation on the column - note you only need ravel if there is one column
df_test["DC"] = imp.transform(df_test[["DC"]]).ravel()

# Deal with any other missing data by deleting the entire row 
df_test.dropna(inplace=True)

# Remove outliers
df_test = outlier_removal(df_test, 120)

# Perform the normalization using the previously defined standard scaler
df_test["DC"] = normalizer.transform(df_test[["DC"]]).ravel()

# Add the "did it rain" feature
df_test = add_did_it_rain(df_test)

# extract the test features and the test target
X_test = df_test.drop("area", axis=1)
y_test = df_test["area"]

In [ ]:
# Use the model to make some predictions
predictions = model.predict(X_test)

In [ ]:
# Score the model on the test set
score = model.score(X_test, y_test)

In [ ]:
print(score)

Conclusion - this model is exceptionally bad - it is worse than just predicting the same value for all elements

### Let's now look at a Random Forest Regression

Note: that these models have hyperparameters, so we are going to have to use cross-validation to optimize our model

Note: we should perform the dataset manipulation in the cross-validation loop. However, since we have a separate test
dataset, and we are only using this to optimize our model this is unlikely to matter too much in practice

In [ ]:
# Instantiate a Random Forest Regression model 
rf_model = RandomForestRegressor(n_estimators=100, max_depth=10)

# Run cross-validation on this model using the training set
# We feed in the model, our training set and the number of folds we want to use
scores = cross_validate(rf_model, X_train, y_train, cv=5, scoring=["r2", "neg_mean_squared_error"])

print(scores)

We can run through a bunch of hyperparameters to see which the best ones are

This is called a **gridsearch**

There is a module in SKLearn that can do this for us automatically, but let's just do it manually for now

In [ ]:
# Set up a grid of hyperparameters to choose from
tree_grid = [100, 200, 300]
depth_grid = [1, 5, 10, 20]

# Initialize a dictionary data structure to store the results
results = {}

for n_trees in tree_grid:
    for depth in depth_grid:
        rf_model = RandomForestRegressor(n_estimators=n_trees, max_depth=depth)
        result = cross_validate(rf_model, X_train, y_train, cv=5, scoring=["r2", "neg_mean_squared_error"])
        # Compute the average results
        r2 = np.mean(result["test_r2"])
        RMSE = np.sqrt(-1 * np.mean(result["test_neg_mean_squared_error"]))
        
        # update the dictionary - the key will be a tuple of hyperparameters, the value a tuple of metrics
        results[(n_trees, depth)] = (r2, RMSE)

In [ ]:
for key, value in results.items():
    print(key, value)

In [ ]:
# Peform the final validation - notice a model can be trained and tested in 3 lines of code, if desired
rf_final_model = RandomForestRegressor(n_estimators=100, max_depth=1)
rf_final_model.fit(X_train, y_train)
predictions = rf_final_model.predict(X_test)
score = rf_final_model.score(X_test, y_test)